This version is implemented based on Sage Math software

In [1]:
import logging
import numpy as np
from functools import reduce
logging.basicConfig(level=logging.DEBUG)

p=random_prime(2^12)
Zp = GF(p)  # Finite Field of size 67

In [2]:
# to store vectorlist for mapping from alphabet to vectors
# alphabetlist='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz '
alphabetlist='ABCD'
vectorlist = matrix(len(alphabetlist))
for i in range(len(alphabetlist)):
   vectorlist[i,i] = 1

#input: alphabet
#ouput: vector
#53 symbols to vectors. a-z,A-Z,and blank.
def MapAlphabetToVector(x):
    i=alphabetlist.index(x)
    return vectorlist[i]

In [3]:
#find a number in a list
#not find, return -1
def FindinList(tem, lxs):
    lenlist=len(lxs)
    ret=-1
    for x in range(lenlist):
        if lxs[x]==tem:
            ret=x
            break
    return ret

In [4]:
#generate n randome numbers, not repeated
def GenR(n, Zp):
    l_xs=[]
    tem = Zp.random_element() # Get next primary number in FP
    while tem == 0:
        tem = Zp.random_element()
    l_xs.append(tem)
    while len(l_xs) < n:
        tem = Zp.random_element()
        while tem==0:
            tem = Zp.random_element()
        if FindinList(tem,l_xs)<0:
            l_xs.append(tem)
    return l_xs


In [5]:
#input:prime number, order
#output a efficiency list of a polynomial
def GenPolynomial(p, n_Polynomialorder):
    Zp = GF(p)
    n_efficiency = []
    for i in range(n_Polynomialorder + 1):
        n_efficiency.append(Zp.random_element())
    return n_efficiency

In [6]:
#input: order, efficiency, xvlaues
#output: Result
def EvalPolynomial(n_Polynomialorder, n_efficiency, xvlaues):
    result=0
    for i in range(n_Polynomialorder + 1):
        result=result + n_efficiency[i] * xvlaues^i
    return result

In [7]:
# Create a secret y among n_clounds with polynomial degree n_order
def secretshare(y, n_clouds, n_order, l_xs):
    shares = []
    f_poly = GenPolynomial(p, n_order)
    f_poly[0] = y
    for x in range(n_clouds):
        resultValue = EvalPolynomial(n_order, f_poly, l_xs[x])
        shares.append(resultValue)
    return shares

In [8]:
def interpolate(x_values, y_values, x=0):
    def _basis(j):
        p = [(x - x_values[m]) / (x_values[j] - x_values[m]) for m in range(k) if m != j]
        return reduce(np.multiply, p)

    assert len(x_values) != 0 and (len(x_values) == len(y_values)), \
        "x and y cannot be empty and must have the same length"
    k = len(x_values)
    return sum(_basis(j) * y_values[j] for j in range(k))

In [9]:
logging.info('main program\n')
logging.info('----------------------------------------------------------------------')
# main program

INFO:root:main program

INFO:root:----------------------------------------------------------------------


In [10]:
# number of clouds
n_clouds=10
# genearte n_clouds x values for each cloud as an identification of an clound
l_xs=[]
l_xs=GenR(n_clouds, Zp)
logging.info(f"The finite filed we are using: Zp={Zp}")
logging.info(f"cloud numbers: {len(l_xs)}")
logging.info(f"x values for each cloud: {l_xs}")

INFO:root:The finite filed we are using: Zp=Finite Field of size 1697
INFO:root:cloud numbers: 10
INFO:root:x values for each cloud: [396, 965, 102, 784, 1169, 1426, 215, 8, 1644, 1320]


In [11]:
type(n_clouds)

<class 'sage.rings.integer.Integer'>

In [12]:
type(l_xs[0])

<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>

In [13]:
# input one file to share
FiletoBestored='ABCD'.strip()
logging.info(f"file stream to be searched: {FiletoBestored}")
# Map FiletoBestored to VectorList
vector=[]

# for temporary store vector
l_Filshares=[]
# for one file in one cloud, one row for one cloud. (for example 10 clouds)
# in each row, every cell for one alphabet (for example each cell consists of 53 elements)

for x in range(n_clouds):
    l_Filshares.append([])

INFO:root:file stream to be searched: ABCD


In [14]:
shares=secretshare(1, n_clouds, 1, l_xs)  # one degree sharing, so we need 1 here.
shares

[838, 1385, 448, 1041, 582, 161, 494, 635, 1316, 1094]

In [15]:
x = FiletoBestored[0]
vector=MapAlphabetToVector(x) # Get a vector representation for a char x
logging.info(f"The char [{x}] --> {vector}")

INFO:root:The char [A] --> (1, 0, 0, 0)


In [16]:
l_y_vectorshares=[] # for one vector,or for one symbol
for xx in range(n_clouds):
    l_y_vectorshares.append([])
l_y_vectorshares

[[], [], [], [], [], [], [], [], [], []]

In [17]:
# for xx in range(n_clouds):
#     l_y_vectorshares[xx].append(shares[xx])
# l_y_vectorshares

In [18]:
for x in FiletoBestored:
    vector=MapAlphabetToVector(x) # Get a vector representation for a char x

    #secret share each vector
    l_y_vectorshares=[] # for one vector,or for one symbol
    for xx in range(n_clouds):
        l_y_vectorshares.append([])

    shares=[]  #for one element (0 or 1)
    for y in vector:
        shares=secretshare(y, n_clouds, 1, l_xs)  # one degree sharing, so we need 1 here.
        for xx in range(n_clouds):
            l_y_vectorshares[xx].append(shares[xx])
    
    # send to clouds
    # transfer data in l_y_vectorshares to l_Filshares
    for xx in range(n_clouds):
        l_Filshares[xx].append(l_y_vectorshares[xx])


In [19]:
logging.info(f"len(l_Filshares) = {len(l_Filshares)}, len(l_Filshares[0]) = {len(l_Filshares[0])} , len(l_Filshares[0][0]) = {len(l_Filshares[0][0])}")

logging.info(f"l_Filshares[0][0]: {l_Filshares[0][0]}")

INFO:root:len(l_Filshares) = 10, len(l_Filshares[0]) = 4 , len(l_Filshares[0][0]) = 4
INFO:root:l_Filshares[0][0]: [520, 530, 769, 80]


In [20]:
logging.info('distribute file ending...')
logging.info('-----------------------------------')

INFO:root:distribute file ending...
INFO:root:-----------------------------------


In [21]:
logging.info('Generate AA now')

n_node=5
l_Node=[]
l_NodesharesforC=[]
for i in range(n_node):
    l_Node.append(0)

INFO:root:Generate AA now


In [22]:
l_Node

[0, 0, 0, 0, 0]

In [23]:
# set initial value for each node
l_Node[0]=1  #for 'L'
l_Node[1]=0  #for 'o'
l_Node[2]=0  #for 'v'
l_Node[3]=0  #for 'e'
l_Node[4]=0

In [24]:
l_Node

[1, 0, 0, 0, 0]

In [25]:
# set positon for the word 'Love'
n_1to2=alphabetlist.index('A') #it is 11
n_2to3=alphabetlist.index('B') #it is 40
n_3to4=alphabetlist.index('C') #it is 47
n_4to5=alphabetlist.index('D') #it is 30

In [26]:
logging.info(f"Index [A -> {n_1to2}], [B -> {n_2to3}], [C -> {n_3to4}], [D -> {n_4to5}]")

INFO:root:Index [A -> 0], [B -> 1], [C -> 2], [D -> 3]


In [27]:
#set initial cell for n_clouds clouds
for xx in range(n_clouds):
    l_NodesharesforC.append([])

In [28]:
l_NodesharesforC

[[], [], [], [], [], [], [], [], [], []]

In [29]:
# secret share AA
# each share for x cloud is stored in l_NodesharesforC[x]
for i in range(n_node):
    shares = secretshare(l_Node[i], n_clouds, i + 2, l_xs)
    logging.info(f"Secret[{l_Node[i]}], Node[{i}], Shares[{i+2}]: {shares}")
    for xx in range(n_clouds):
#         logging.info(f"Secret[{l_Node[i]}], Cloud[{xx}], Shares: {shares[xx]}")
        l_NodesharesforC[xx].append(shares[xx])

INFO:root:Secret[1], Node[0], Shares[2]: [1583, 683, 489, 1026, 1302, 387, 1506, 324, 574, 507]
INFO:root:Secret[0], Node[1], Shares[3]: [1149, 809, 386, 962, 136, 123, 1432, 1441, 19, 165]
INFO:root:Secret[0], Node[2], Shares[4]: [320, 432, 500, 760, 835, 1665, 174, 763, 1246, 275]
INFO:root:Secret[0], Node[3], Shares[5]: [1082, 276, 841, 1416, 1282, 225, 579, 1032, 978, 1646]
INFO:root:Secret[0], Node[4], Shares[6]: [1091, 1211, 1367, 34, 964, 1297, 482, 1394, 713, 1599]


In [30]:
logging.info(f"len(l_NodesharesforC) = {len(l_NodesharesforC)}, len(l_NodesharesforC[0]) = {len(l_NodesharesforC[0])}")

logging.info(f"l_NodesharesforC[0]: {l_NodesharesforC[0]}")

INFO:root:len(l_NodesharesforC) = 10, len(l_NodesharesforC[0]) = 5
INFO:root:l_NodesharesforC[0]: [1583, 1149, 320, 1082, 1091]


The state of each node will be shared among 10 clounds, as shown in l_NodesharesforC. In this object, each row means five shares state of nodes in a cloud server.

In [31]:
l_NodesharesforC

[[1583, 1149, 320, 1082, 1091],
 [683, 809, 432, 276, 1211],
 [489, 386, 500, 841, 1367],
 [1026, 962, 760, 1416, 34],
 [1302, 136, 835, 1282, 964],
 [387, 123, 1665, 225, 1297],
 [1506, 1432, 174, 579, 482],
 [324, 1441, 763, 1032, 1394],
 [574, 19, 1246, 978, 713],
 [507, 165, 275, 1646, 1599]]

In [32]:
logging.info(f"original AA:N0,N1,N2,N3,N4 -> {l_Node}")
logging.info(f"Generate AA ending...")
logging.info("---------------------------------------")

INFO:root:original AA:N0,N1,N2,N3,N4 -> [1, 0, 0, 0, 0]
INFO:root:Generate AA ending...
INFO:root:---------------------------------------


In [33]:
# todo upate AA in clouds.
logging.info("Updating AA now")

# each cloud will update AA for theirselves
FileLen=len(l_Filshares[0]) # it is 14 for 'Alice Love Bob'
FileLen

INFO:root:Updating AA now


4

In [34]:
for i in range(FileLen):
    # for each symbol in file stream
    for xx in range(n_clouds):
        logging.info(f"[B]-Server[{xx}, Input {l_Filshares[xx][i]}, State Machine {l_NodesharesforC[xx]}")
        logging.info(f"[4]-{l_NodesharesforC[xx][4]} = {l_NodesharesforC[xx][4]} + {l_NodesharesforC[xx][3]} * {l_Filshares[xx][i][n_4to5]}")
        l_NodesharesforC[xx][4] = l_NodesharesforC[xx][4] + l_NodesharesforC[xx][3] * l_Filshares[xx][i][n_4to5]
        logging.info(f"[4]-{l_NodesharesforC[xx][4]}")
        
        
        
        
        
        logging.info(f"[3]-{l_NodesharesforC[xx][3]} = {l_NodesharesforC[xx][2]} * {l_Filshares[xx][i][n_3to4]}")
        l_NodesharesforC[xx][3] = l_NodesharesforC[xx][2] * l_Filshares[xx][i][n_3to4]
        logging.info(f"[3]-{l_NodesharesforC[xx][3]}")
        
        logging.info(f"[2]-{l_NodesharesforC[xx][2]} = {l_NodesharesforC[xx][1]} * {l_Filshares[xx][i][n_2to3]}")
        l_NodesharesforC[xx][2] = l_NodesharesforC[xx][1] * l_Filshares[xx][i][n_2to3]
        logging.info(f"[2]-{l_NodesharesforC[xx][2]}")
        
        logging.info(f"{l_NodesharesforC[xx][1]} = {l_NodesharesforC[xx][0]} * {l_Filshares[xx][i][n_1to2]}")
        l_NodesharesforC[xx][1] = l_NodesharesforC[xx][0] * l_Filshares[xx][i][n_1to2]
        logging.info(f"[1]-{l_NodesharesforC[xx][1]}")
        
        logging.info(f"[A]-Server[{xx}, Input {l_Filshares[xx][i]}, State Machine {l_NodesharesforC[xx]}\n")

INFO:root:[B]-Server[0, Input [520, 530, 769, 80], State Machine [1583, 1149, 320, 1082, 1091]
INFO:root:[4]-1091 = 1091 + 1082 * 80
INFO:root:[4]-1104
INFO:root:[3]-1082 = 320 * 769
INFO:root:[3]-15
INFO:root:[2]-320 = 1149 * 530
INFO:root:[2]-1444
INFO:root:1149 = 1583 * 520
INFO:root:[1]-115
INFO:root:[A]-Server[0, Input [520, 530, 769, 80], State Machine [1583, 115, 1444, 15, 1104]

INFO:root:[B]-Server[1, Input [1510, 923, 1304, 1292], State Machine [683, 809, 432, 276, 1211]
INFO:root:[4]-1211 = 1211 + 276 * 1292
INFO:root:[4]-1433
INFO:root:[3]-276 = 432 * 1304
INFO:root:[3]-1621
INFO:root:[2]-432 = 809 * 923
INFO:root:[2]-27
INFO:root:809 = 683 * 1510
INFO:root:[1]-1251
INFO:root:[A]-Server[1, Input [1510, 923, 1304, 1292], State Machine [683, 1251, 27, 1621, 1433]

INFO:root:[B]-Server[2, Input [1446, 1165, 1098, 432], State Machine [489, 386, 500, 841, 1367]
INFO:root:[4]-1367 = 1367 + 841 * 432
INFO:root:[4]-1521
INFO:root:[3]-841 = 500 * 1098
INFO:root:[3]-869
INFO:root:[2]

INFO:root:[B]-Server[1, Input [1078, 873, 581, 1407], State Machine [683, 787, 1570, 435, 602]
INFO:root:[4]-602 = 602 + 435 * 1407
INFO:root:[4]-30
INFO:root:[3]-435 = 1570 * 581
INFO:root:[3]-881
INFO:root:[2]-1570 = 787 * 873
INFO:root:[2]-1463
INFO:root:787 = 683 * 1078
INFO:root:[1]-1473
INFO:root:[A]-Server[1, Input [1078, 873, 581, 1407], State Machine [683, 1473, 1463, 881, 30]

INFO:root:[B]-Server[2, Input [434, 87, 1645, 875], State Machine [489, 1577, 1510, 1012, 1139]
INFO:root:[4]-1139 = 1139 + 1012 * 875
INFO:root:[4]-805
INFO:root:[3]-1012 = 1510 * 1645
INFO:root:[3]-1239
INFO:root:[2]-1510 = 1577 * 87
INFO:root:[2]-1439
INFO:root:1577 = 489 * 434
INFO:root:[1]-101
INFO:root:[A]-Server[2, Input [434, 87, 1645, 875], State Machine [489, 101, 1439, 1239, 805]

INFO:root:[B]-Server[3, Input [1173, 1068, 1457, 969], State Machine [1026, 790, 1043, 553, 1162]
INFO:root:[4]-1162 = 1162 + 553 * 969
INFO:root:[4]-767
INFO:root:[3]-553 = 1043 * 1457
INFO:root:[3]-836
INFO:root:[

In [35]:
1910 * 2026

3869660

In [36]:
type(l_NodesharesforC[0][0])

<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>

In [37]:
logging.info("The state machine is updated according to input")
l_NodesharesforC

INFO:root:The state machine is updated according to input


[[1583, 443, 943, 1686, 941],
 [683, 254, 1546, 608, 1372],
 [489, 537, 1487, 132, 701],
 [1026, 283, 411, 450, 1509],
 [1302, 1268, 1118, 817, 574],
 [387, 905, 288, 1271, 583],
 [1506, 792, 1149, 303, 1619],
 [324, 72, 1223, 474, 682],
 [574, 192, 1586, 1598, 1439],
 [507, 1620, 64, 1319, 266]]

In [38]:
logging.info("Upate AA ending...")
logging.info("--------------------------------------------")

# todo Reconstruct AA
logging.info("Reconstruct AA now")
# From shares got from clouds
# which are stored in list l_NodesharesforC
n_node = 5
for i in range(n_node):
    l_Node[i] = 0
logging.info(f"l_Node is reinitilized {l_Node}")

INFO:root:Upate AA ending...
INFO:root:--------------------------------------------
INFO:root:Reconstruct AA now
INFO:root:l_Node is reinitilized [0, 0, 0, 0, 0]


In [39]:
logging.info(f"The identifier of each party server is: l_xs={l_xs}")
for xx in range(5):
    lxs = []
    lys = []
    for y in range(xx + 3):
        lxs.append(l_xs[y])
        lys.append(l_NodesharesforC[y][xx])
    
    l_Node[xx] = interpolate(lxs, lys)
    logging.info(f"l_Node[{xx}] = {l_Node[xx]} is recovered using lxs={lxs}, lys={lys}")


INFO:root:The identifier of each party server is: l_xs=[396, 965, 102, 784, 1169, 1426, 215, 8, 1644, 1320]
INFO:root:l_Node[0] = 1 is recovered using lxs=[396, 965, 102], lys=[1583, 683, 489]
INFO:root:l_Node[1] = 0 is recovered using lxs=[396, 965, 102, 784], lys=[443, 254, 537, 283]
INFO:root:l_Node[2] = 0 is recovered using lxs=[396, 965, 102, 784, 1169], lys=[943, 1546, 1487, 411, 1118]
INFO:root:l_Node[3] = 0 is recovered using lxs=[396, 965, 102, 784, 1169, 1426], lys=[1686, 608, 132, 450, 817, 1271]
INFO:root:l_Node[4] = 1 is recovered using lxs=[396, 965, 102, 784, 1169, 1426, 215], lys=[941, 1372, 701, 1509, 574, 583, 1619]


In [40]:
# the second node

logging.info(f"Reconstruct AA:N0,N1,N2,N3,N4 -> {l_Node}")
logging.info("Reconstruct AA ending...")
logging.info("-------------------------------------")

INFO:root:Reconstruct AA:N0,N1,N2,N3,N4 -> [1, 0, 0, 0, 1]
INFO:root:Reconstruct AA ending...
INFO:root:-------------------------------------
